# RÃ©solution problÃ¨me Teneq1b

In [ ]:
using ForwardDiff, JuMP, Ipopt

## RÃ©solution avec ENLSIP-Julia

In [ ]:
include("../src/enlsip_functions_castor2.jl")

In [ ]:
n = 10
m = 10
nb_eq = 0
nb_constraints = 10

In [ ]:
# Residuals 

function r(x::AbstractArray)
    R = 40
    sum_x = sum(x)
    rx = [x[1]+x[4]-3;
        2*x[1]+x[2]+x[4]+x[7]+x[8]+x[9]+2*x[10]-R;
        2*x[2]+2*x[5]+x[6]+x[7]-8;
        2*x[3]+x[5]-4*R;
        x[1]*x[5]-0.193*x[2]*x[5];
        x[6]*sqrt(x[2])- 0.002597*sqrt(x[2]*x[4]*sum_x);
        x[7]*sqrt(x[4])- 0.003448*sqrt(x[1]*x[4]*sum_x);
        x[8]*x[4]-0.00001799*x[2]*sum_x;
        x[9]*x[4]-0.0002155*x[1]*sqrt(x[3]*sum_x);
        x[10]*(x[4]^2) - 0.00003846*(x[4]^2) * sum_x]
    return rx
end



resT1b = ResidualsEval(0)

function (resT1b::ResidualsEval)(x::Vector, rx::Vector, J::Matrix)

    # Evaluate the residuals
    if abs(resT1b.ctrl) == 1
        rx[:] = r(x)

    # The jacobian is computed analytically
    elseif resT1b.ctrl == 2
        J[:] = ForwardDiff.jacobian(r,x)
    end
    return
end

In [ ]:
# Constraints
function c(x::AbstractArray)
    res = [x[1]-0.0001;
        x[2]-0.0001;
        x[3]-0.0001;
        x[4]-0.0001;
        x[5];
        x[6];
        x[7];
        x[8];
        x[9];
        x[10]]
    return res
end

consT1b = ConstraintsEval(0)

function (consT1b::ConstraintsEval)(x::Vector, cx::Vector, A::Matrix)

    # Evaluate the constraints
    if abs(consT1b.ctrl) == 1
        cx[:] = c(x)
    # The jacobian is computed numerically if ctrl is set to 0 on return
    elseif consT1b.ctrl == 2
        A[:] = ForwardDiff.jacobian(c,x)
    end
    return
end

In [ ]:
x0 = [1.;1.;20.;1.;0.;0.;0.;0.;0.;1.] 
# x0 = [2.,5.,40.,1.,0.,0.,0.,0.,0.,5.]
e = eps(Float64)
se = sqrt(e)
enlsipTeneq1b = enlsip(x0,resT1b,consT1b,n,m,nb_eq,nb_constraints)

In [ ]:
# x0 = [1.;1.;20.;1.;0.;0.;0.;0.;0.;1.] 
x0 = [2.,5.,40.,1.,0.,0.,0.,0.,0.,5.]
e = eps(Float64)
se = sqrt(e)
enlsipTeneq1b = enlsip(x0,resT1b,consT1b,n,m,nb_eq,nb_constraints)

In [ ]:
enlsipTeneq1b.sol

## RÃ©solution avec Ipopt

In [ ]:
R = 40
x0 = [1.;1.;20.;1.;0.;0.;0.;0.;0.;1.] 
# x0 = [2.,5.,40.,1.,0.,0.,0.,0.,0.,5.]
x_low = [0.0001,0.0001,0.0001,0.0001,0.,0.,0.,0.,0.,0.]
x_upp = Inf*ones(n)

model = Model(Ipopt.Optimizer)

@variable(model, x_low[i] <= x[i=1:n] <= x_upp[i], start = x0[i])

@NLobjective(model, Min, (x[1]+x[4]-3)^2+
        (2*x[1]+x[2]+x[4]+x[7]+x[8]+x[9]+2*x[10]-R)^2+
        (2*x[2]+2*x[5]+x[6]+x[7]-8)^2+
        (2*x[3]+x[5]-4*R)^2+
        (x[1]*x[5]-0.193*x[2]*x[5])^2+
        (x[6]*sqrt(x[2])- 0.002597*sqrt(x[2]*x[4]*sum(x[i] for i=1:n)))^2+
        (x[7]*sqrt(x[4])- 0.003448*sqrt(x[1]*x[4]*sum(x[i] for i=1:n)))^2+
        (x[8]*x[4]-0.00001799*x[2]*sum(x[i] for i=1:n))^2+
        (x[9]*x[4]-0.0002155*x[1]*sqrt(x[3]*sum(x[i] for i=1:n)))^2+
        (x[10]*(x[4]^2) - 0.00003846*(x[4]^2) * sum(x[i] for i=1:n))^2)

optimize!(model)

value.(model[:x])